In [6]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
 
interactions_df = pd.read_excel('Combined_Interactions.xlsx')
events_df = pd.read_excel('Events.xlsx')
responses = pd.read_csv('(Responses) (1).csv')
 
item_user_matrix = interactions_df.pivot_table(index='Event ID', columns='Name', values='Interaction Score', fill_value=0)
item_similarity = cosine_similarity(item_user_matrix)
 
def item_based_recommend_events(user, top_n=5):
    # Get the events already interacted with by the user
    interacted_events = interactions_df[interactions_df['Name'] == user]['Event ID'].tolist()
    # Initialize dictionary to store event recommendations
    event_recommendations = {}
 
    # Iterate over the interacted events and recommend similar events
    for event_id in interacted_events:
        if event_id not in item_user_matrix.index:
            continue
        event_index = item_user_matrix.index.get_loc(event_id)
        similar_events = np.argsort(item_similarity[event_index])[::-1]  # Sort events by similarity
        for similar_event in similar_events:
            similar_event_id = item_user_matrix.index[similar_event]
            if similar_event_id not in interacted_events:
                if similar_event_id not in event_recommendations:
                    event_recommendations[similar_event_id] = item_similarity[event_index][similar_event]
                else:
                    event_recommendations[similar_event_id] += item_similarity[event_index][similar_event]
    # Sort recommendations by similarity score
    sorted_recommendations = sorted(event_recommendations.items(), key=lambda x: x[1], reverse=True)
    # Return top n recommendations
    return sorted_recommendations[:top_n]
 
# Hybrid recommendation function
def recommend_events(user, events_df, responses, top_n=5):
    ##### Collaborative Based Filtering using Item Similarity
    item_based_recommendations = item_based_recommend_events(user, top_n)
    recommended_events = pd.DataFrame(item_based_recommendations, columns=['Event ID', 'score'])
 
    # Merge with events_df to get event details
    recommended_events = recommended_events.merge(events_df, on='Event ID')
    ##### Content Based Filtering
    responses = responses[responses['Name'] == user]
    responses['All Interests'] = responses['All Interests'].str.split(', ')
    events_df['Category'] = events_df[['Primary Category', 'Secondary Category']].apply(lambda x: ', '.join(filter(lambda y: pd.notna(y), x)), axis=1)
    events_df['Category'] = events_df['Category'].str.split(', ')
    events_df['Interest_Match'] = False
 
    for index, row in responses.iterrows():
        for tag in row['All Interests']:
            events_df.loc[events_df['Category'].apply(lambda x: tag in x), 'Interest_Match'] = True
 
    events_df = events_df[events_df['Interest_Match'] == True]
 
    interacted_events = interactions_df[interactions_df['Name'] == user]
    filtered_interactions_df_1 = interacted_events[~interacted_events['Interaction Score'].isin([-1, 0])]
    merged_df = pd.merge(interacted_events, events_df[['Event ID', 'Primary Category', 'Sub Category']], on='Event ID', how='left', suffixes=('_int', '_evt'))
    merged_df_1 = pd.merge(filtered_interactions_df_1, events_df[['Event ID', 'Primary Category', 'Sub Category']], on='Event ID', how='left', suffixes=('_int', '_evt'))
    filtered_interactions_df_neg_zero = interacted_events[interacted_events['Interaction Score'].isin([-1, 0])]
 
    categories = merged_df.sort_values('Interaction Score', ascending=False)['Primary Category_evt'].unique()
    recommended_categories = set()
 
    for category in categories:
        category_events = events_df[events_df['Primary Category'] == category]
        user_subcategories = merged_df_1[merged_df_1['Primary Category_evt'] == category]['Sub Category_evt'].unique()
        for subcategory in user_subcategories:
            subcategory_events = category_events[category_events['Sub Category'] == subcategory]
            new_events = subcategory_events[~subcategory_events['Event ID'].isin(merged_df_1['Event ID']) &
                                            ~subcategory_events['Event ID'].isin(recommended_events['Event ID']) &
                                            ~subcategory_events['Event ID'].isin(filtered_interactions_df_neg_zero['Event ID'])]
            if not new_events.empty:
                recommended_categories.add(category)
                recommended_events = pd.concat([recommended_events, new_events.head(1)], ignore_index=True)
 
    # Add unrecommended events
    unrecommended_categories = set(categories) - recommended_categories
    for ur_category in unrecommended_categories:
        category_events_1 = events_df[events_df['Primary Category'] == ur_category]
        user_subcategories_1 = merged_df[merged_df['Primary Category_evt'] == ur_category]['Sub Category_evt'].unique()
        for sc in user_subcategories_1:
            subcategory_events_1 = category_events_1[category_events_1['Sub Category'] != sc]
            ur_category_events = subcategory_events_1[
                (~subcategory_events_1['Event ID'].isin(merged_df['Event ID'])) &
                (~subcategory_events_1['Event ID'].isin(recommended_events['Event ID']))]
            recommended_events = pd.concat([recommended_events, ur_category_events.head(1)], ignore_index=True)
 
    # Add additional events based on sorting
    sorted_events = merged_df.sort_values('Interaction Score', ascending=False)
    for index, event in sorted_events.iterrows():
        additional_events = events_df[
            (events_df['Primary Category'] == event['Primary Category_evt']) &
            (~events_df['Event ID'].isin(merged_df['Event ID'])) &
            (~events_df['Event ID'].isin(recommended_events['Event ID']))]
        recommended_events = pd.concat([recommended_events, additional_events.head(1)], ignore_index=True)
 
    recommended_events.drop('score', axis=1, inplace=True)
    return recommended_events
 
user = 'Pratik Bhande'
recommended_events = recommend_events(user, events_df, responses)
recommended_events.head(10)

C:\Users\pratik\AppData\Local\Temp\ipykernel_17444\4277876826.py:46: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  responses['All Interests'] = responses['All Interests'].str.split(', ')


,Event ID,Event Names,Description,Primary Category,Secondary Category,Sub Category,Location,Prices,Date,Unnamed: 9,Time,Links,Category,Interest_Match
0,EVT0030,Matheran Forest Camping,NaN,Travel,NaN,Camping,Sangamvadi,1499,2024-02-06 00:00:00,Tuesday,19:00:00,NaN,NaN,NaN
1,EVT0085,Dandeli Jungle Tour,उनाड भ्रमंती is happy to invite you for *Dande...,Adventure,Travel,Hiking and Trekking,Sangamvadi,10000,2024-03-29 00:00:00,Friday,00:00:00,https://allevents.in/pune/dandeli-jungle-tour-...,NaN,NaN
2,EVT0122,NutriWell Business Conclave,Nutriwell Business Conclave organised by ETHea...,Fitness,NaN,Nutrition,Bavdhan,499,2024-07-05 00:00:00,NaN,05:00:00,NaN,NaN,NaN
3,EVT0147,Flea Market - local shopping,Explore innovative startups and entrepreneurs ...,Music and Entertainment,Food and Drinks,Shopping,Karve Nagar,50,2024-04-13 00:00:00,NaN,17:00:00,NaN,NaN,NaN
4,EVT0022,Fun and Fair,"Don’t miss new attractions, the thrills of the...",Music and Entertainment,Food and Drinks,Exhibition,Erandwane,0,23/3/24 to 31/3/24,23/3/24 to 31/3/24,15:00:00,NaN,NaN,NaN
5,EVT0081,Kolad River Side Camping and Water Sports,*Dear people* Adventure Camp Zone ( Kolad )Get...,Adventure,Travel,Adventure Sports,Deccan,1500,2024-03-31 00:00:00,Sunday,15:00:00,https://allevents.in/roha/kolad-river-side-cam...,"[Adventure, Travel]",True
6,EVT0132,Climb Koleshwar Plateau,"Pitch your tents, Dinner and Stay near Jambhal...",Adventure,NaN,Hiking and Trekking,Sangamvadi,2000,2024-04-30 00:00:00,NaN,06:00:00,NaN,[Adventure],True
7,EVT0052,Friday Diva Night - DJ Veronica,CELEBRATE FRIDAY DIVA NIGHT WITH CHEERS TO YOU...,Music and Entertainment,NaN,Concert,Koregaon Park,2500,2024-11-18 00:00:00,Monday,18:00:00,https://allevents.in/pune/friday-diva-night-dj...,[Music and Entertainment],True
8,EVT0068,Theatre Shows,Join us for an enchanting Theatre Workshop des...,Music and Entertainment,NaN,Drama,Deccan,NaN,Multiple Dates,Multiple Dates,10:00:00,NaN,[Music and Entertainment],True
9,EVT0031,Pharma and Lab Expo,NaN,Tech and Innovation,NaN,Exhibition,Kharadi,400,2024-07-14 00:00:00,Sunday,17:00:00,NaN,[Tech and Innovation],True


### Code Flow and Explanation:

1. **Import Libraries**:
   - `pandas` for data manipulation.
   - `cosine_similarity` from `sklearn.metrics.pairwise` for calculating item similarity.
   - `numpy` for numerical operations.

2. **Load Data**:
   - Load the user-event interaction data (`Combined_Interactions.xlsx`), event details (`Events.xlsx`), and user responses (`(Responses) (1).csv`) into dataframes.

3. **Item-User Matrix Creation**:
   - Create an **item-user interaction matrix** where rows are event IDs, columns are users, and values are interaction scores. Missing values are filled with 0.

4. **Item Similarity Calculation**:
   - Use **cosine similarity** to calculate the similarity between events based on user interactions. This forms an **item-item similarity matrix**.

5. **Item-Based Recommendation Function**:
   - The function `item_based_recommend_events()` recommends similar events based on events the user has interacted with:
     - Identify events that the user has already interacted with.
     - For each interacted event, find similar events using the **item-item similarity matrix**.
     - Ensure that recommended events have not been interacted with by the user.
     - Return the top `n` event recommendations based on similarity scores.

6. **Hybrid Recommendation Function**:
   - The `recommend_events()` function provides a hybrid recommendation system combining **item-based collaborative filtering** and **content-based filtering**:
     - **Collaborative Filtering**: Uses the item-based recommendations generated by `item_based_recommend_events()`.
     - **Content-Based Filtering**:
       - Match the user's interests (from the responses dataset) to event categories.
       - Filter events based on whether their categories match the user's interests.
       - Recommend new events from categories or subcategories the user has not interacted with.
     - The recommendations are sorted and merged with event details from `events_df`.

7. **Unrecommended Events**:
   - If certain categories are underrepresented in the recommendations, the function identifies additional events from unrecommended categories to ensure variety.

8. **Final Event Selection**:
   - Additional events are selected based on user interaction scores, and the final set of recommendations is returned.

9. **Example Usage**:
   - The function is called for the user `Pratik Bhande` to generate event recommendations, and the top 10 recommended events are displayed.